In [1]:
import os
import cv2
import numpy as np
import math
import face_recognition
import pyttsx3
from keras.models import load_model
from keras.utils import CustomObjectScope
import imutils
import mediapipe as mp 
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.metrics import accuracy_score  
import pickle
import time
import dlib


Speech Engine Functionn

In [2]:
engine=pyttsx3.init()

#Text to speech 
def speak(audio):
    start_time = time.time()
    engine.say(audio)
    engine.runAndWait()
    latency = time.time() - start_time
   
    print("Latency: " + str(latency))

    engine.stop(
    
    )


Load Models

In [3]:

with CustomObjectScope({'acc': ['acc']}):
    model1 = load_model("C:/Users/MANSI YAGNIK/facerecog/face/mask.h5")
labels_dict = {0: 'without mask', 1: 'mask'}
focal_length = 800
object_width = 200

In [4]:

predictor_path = ('C:/Users/MANSI YAGNIK/facerecog/face/shape_predictor_68_face_landmarks.dat')
predictor = dlib.shape_predictor(predictor_path)
face_cascade = cv2.CascadeClassifier("face\haarcascade_frontalface_default.xml")
detector = dlib.get_frontal_face_detector()

Extracting landmarks

In [5]:
path = 'C:/Users/MANSI YAGNIK/facerecog/face/faces'

def findEncodings(path,cl):
    image = os.path.join(path,cl)
    reference_image = cv2.imread(image)
    reference_gray = cv2.cvtColor(reference_image, cv2.COLOR_BGR2GRAY)
    reference_landmarks1 = predictor(reference_gray, dlib.rectangle(0, 0, reference_gray.shape[1], reference_gray.shape[0]))
    
    
    left_eye_template = [(reference_landmarks1.part(i).x, reference_landmarks1.part(i).y) for i in range(36, 42)]
    left_eye_template_array = np.array(left_eye_template)
    right_eye_template = [(reference_landmarks1.part(i).x, reference_landmarks1.part(i).y) for i in range(42, 48)]
    right_eye_template_array=np.array(right_eye_template)
    
    left_eyebrow_template=[(reference_landmarks1.part(i).x, reference_landmarks1.part(i).y) for i in range(17, 22)]
    left_eyebrow_template_array=np.array(left_eyebrow_template)
    
    right_eyebrow_template=[(reference_landmarks1.part(i).x, reference_landmarks1.part(i).y) for i in range(22, 27)]
    right_eyebrow_template_array=np.array(right_eyebrow_template)
    
    
    return left_eye_template_array, right_eye_template_array,left_eyebrow_template_array,right_eyebrow_template_array


Confidence Estimation

In [6]:

def faceAccuray(face_distance, face_threshold=0.6):
    range = (1.0 - face_threshold)
    linear_val = (1.0 - face_distance) / (range * 2.0)

    if face_distance > face_threshold:
        

        return str(round(linear_val * 100, 2)) + '%'
    else:
        value = (linear_val + ((1.0 - linear_val) *
                 math.pow((linear_val - 0.5) * 2, 0.2))) * 100
        return str(round(value, 2)) + '%'

Pose detection

In [7]:
mp_drawing = mp.solutions.drawing_utils  # Drawing helpers
mp_holistic = mp.solutions.holistic  # Mediapipe Solutions

df = pd.read_csv('C:/Users/MANSI YAGNIK/facerecog/face/coords1.csv')

df[df['class'] == 'Stand with hands up']

X = df.drop('class', axis=1)  # features
y = df['class']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1234)

pipelines = {
    # 'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc': make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf': make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb': make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

list(pipelines.values())[0]

fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train,y_train)
    fit_models[algo] = model

for algo, model in fit_models.items():
    print(X_test.shape)
    
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

fit_models['rf'].predict(X_test)

with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)


(525, 132)
rc 1.0
(525, 132)
rf 0.9980952380952381
(525, 132)
gb 0.9980952380952381


Real time

In [9]:
class FaceRecognition:
    face_locations = []
    face_encodings = []
    face_names = []
    known_face_encodings = []
    known_face_names = []
    process_current_frame = True

    def __init__(self):
        self.encode_faces()

    def encode_faces(self):
        for image in os.listdir('faces'):
            face_image = face_recognition.load_image_file(f"faces/{image}")
            face_encoding = face_recognition.face_encodings(face_image)[0]

            self.known_face_encodings.append(face_encoding)
            self.known_face_names.append(image)
        print(self.known_face_names)

    def run_recognition(self):
        video_capture = cv2.VideoCapture(0)
        for i in range(10):  # Try a range of indices to find supported resolutions
            width = video_capture.get(3)  # Width
            height = video_capture.get(4)# Height
            print("Resolution")
            print(width)
            print(height)
        body_function=False
        mask_fuction=False
        function_executed=False
        distance_fuc=False
        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
            while True:
                ret, frame = video_capture.read()
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = detector(gray)
                small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
                rgb_small_frame = small_frame[:, :, ::-1]
                gray_frame = cv2.cvtColor(rgb_small_frame, cv2.COLOR_BGR2GRAY)
                normalized_frame = cv2.equalizeHist(gray_frame)
                rgb_frame = cv2.cvtColor(normalized_frame, cv2.COLOR_GRAY2RGB)
                results = holistic.process(rgb_frame)
                self.face_locations = face_recognition.face_locations(
                rgb_frame,number_of_times_to_upsample=3)
                self.face_encodings = face_recognition.face_encodings(
                rgb_frame, self.face_locations)
                
                print(rgb_frame.shape)
                rgb_frame1 = cv2.resize(rgb_frame, (256, 256))
                rgb_frame2 = np.expand_dims(rgb_frame1, axis=0)
                result1 = model1.predict(rgb_frame2)
                label = np.argmax(result1, axis=1)[0]
                for face in faces:
                   landmarks1 = predictor(gray, face)


                   left_eye_landmarks1 = [(landmarks1.part(i).x, landmarks1.part(i).y) for i in range(36, 42)]
                # print(left_eye_landmarks)
                   left_eye_landmarks_array=np.array(left_eye_landmarks1)
                
                   right_eye_landmarks1 = [(landmarks1.part(i).x, landmarks1.part(i).y) for i in range(42, 48)]
                # print(right_eye_landmarks)
                   right_eye_landmarks_array=np.array(right_eye_landmarks1)
                
                   left_eyebrow_landmarks1=[(landmarks1.part(i).x, landmarks1.part(i).y) for i in range(17, 22)]
                   left_eyebrow_landmarks_array=np.array(left_eyebrow_landmarks1)
                
                   rigth_eyebrow_landmarks1=[(landmarks1.part(i).x, landmarks1.part(i).y) for i in range(22, 27)]
                   rigth_eyebrow_landmarks_array=np.array(rigth_eyebrow_landmarks1)
                
                   for cl in os.listdir(path):
                        left_eye_template_array, right_eye_template_array,left_eyebrow_template_array,right_eyebrow_template_array = findEncodings(path, cl)

                        euclidean_distanceLeft = np.linalg.norm(left_eye_landmarks_array - left_eye_template_array)/1000
                        euclidean_distanceright = np.linalg.norm(right_eye_landmarks_array - right_eye_template_array)/1000
                        euclidean_distanceLeft_eyebrow = np.linalg.norm(left_eyebrow_landmarks_array - left_eyebrow_template_array)/1000
                        euclidean_distanceright_eyebrow = np.linalg.norm(rigth_eyebrow_landmarks_array - right_eyebrow_template_array)/1000
                        print(euclidean_distanceright_eyebrow)
                    
                        threshold = 0.6
                
                        if euclidean_distanceLeft > threshold and euclidean_distanceLeft_eyebrow>threshold and euclidean_distanceright > threshold and euclidean_distanceright_eyebrow>threshold:
                           name = os.path.splitext(cl)[0]
                           print('name:', name)
                           
                           cv2.putText(frame, name, (face.left(), face.bottom() ), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
                        
                        
                           print("Eyes matched!")
                        else:
                           print("Eyes not matched.")
                        for landmark in landmarks1.parts():
                           cv2.circle(frame, (landmark.x, landmark.y), 2, (0, 0, 255), -1)

                print(label)
                if not mask_fuction:
                   speak( labels_dict[label])
                   mask_fuction = True
            
            
            
                self.face_names = [] 
            
                for face_encoding in self.face_encodings:
                    matches = face_recognition.compare_faces(
                    self.known_face_encodings, face_encoding)

                # Calculate the shortest distance to face
                    face_distances = face_recognition.face_distance(
                        self.known_face_encodings, face_encoding)
                    best_match_index = np.argmin(face_distances)
                    if matches[best_match_index]:
                        name = self.known_face_names[best_match_index].split('.')[0]
                        confidence = faceAccuray(face_distances[best_match_index])
                    
                    

                    else:
                    # matches = face_recognition.compare_faces(self.known_face_encodings, face_encoding)
                        name = "Unknown"
                        confidence = '???'
                    self.face_names.append(f'{name} ({confidence})')
                
                    if not function_executed:
                        speak(name)
                        function_executed = True
                    self.process_current_frame = not self.process_current_frame
            # Display the results
                for (top, right, bottom, left), name in zip(self.face_locations, self.face_names):
                # Scale back up face locations since the frame we detected in was scaled to 1/4 size
                    top *= 4
                    right *= 4
                    bottom *= 4
                    left *= 4
                    rgb_frame.flags.writeable = True   
                    rgb_frame1 = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)
        # 2. Right hand
                    mp_drawing.draw_landmarks(rgb_frame1, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        
                    mp_drawing.draw_landmarks(rgb_frame1, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
                    mp_drawing.draw_landmarks(rgb_frame1, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
            # Grab ear coords
                    pose = results.pose_landmarks.landmark
                    print(pose)
                    pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

            # Concate rows
                    row = pose_row
            # Make Detections
                    X = pd.DataFrame([row])
                    body_language_class = model.predict(X)[0]
                    body_language_prob = model.predict_proba(X)[0]
                    print(body_language_class, body_language_prob)
                    coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
                    
                    
                    cv2.rectangle(frame, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
                    cv2.putText(frame, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
                    cv2.rectangle(frame, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
                    cv2.putText(frame, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                    cv2.putText(frame, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
                    cv2.putText(frame, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                    cv2.putText(frame, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                
                
                    face_width_pixels =left

        # Calculate the distance to the face based on the size of the face in the image
                    distance = (object_width * focal_length) / face_width_pixels 
                    distance=distance/305
                    if not distance_fuc:
                       speak(distance )
                       speak("feet" )

                       distance_fuc = True
                # Create the frame with the name
                    cv2.rectangle(frame, (left, top),
                              (right, bottom), (0, 0, 255), 2)
                    cv2.rectangle(frame, (left, bottom - 35),
                              (right, bottom), (0, 0, 255), cv2.FILLED)
                    cv2.putText(frame, name, (left + 6, bottom - 6),
                            cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 255, 255), 1)
                    
                    cv2.putText(frame, f"Face Distance: {distance:.2f} feet", (left, top +5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
                cv2.imshow('Face Recognition', frame)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                   break

        # Release handle to the webcam
        video_capture.release()
        cv2.destroyAllWindows()


In [11]:
if __name__ == '__main__':
    fr = FaceRecognition()
    fr.run_recognition()

['Ekagrah Grover.jpg', 'Mehreen Hashmi.png', 'Yash Dhanwariya.png', 'Ekagrah Grover.jpg', 'Mehreen Hashmi.png', 'Yash Dhanwariya.png']
Resolution
640.0
480.0
Resolution
640.0
480.0
Resolution
640.0
480.0
Resolution
640.0
480.0
Resolution
640.0
480.0
Resolution
640.0
480.0
Resolution
640.0
480.0
Resolution
640.0
480.0
Resolution
640.0
480.0
Resolution
640.0
480.0
(120, 160, 3)
1/1 [==============================] - 0s 143ms/step
0.6185547671791075
Eyes not matched.
0.27007406391580807
Eyes not matched.
0.165792038409569
Eyes not matched.
0
Latency: 2.132230520248413
Latency: 1.397097110748291
[x: 0.5010715126991272
y: 0.5401270389556885
z: -1.401929497718811
visibility: 0.9996187686920166
, x: 0.535849928855896
y: 0.46577394008636475
z: -1.3141885995864868
visibility: 0.9991839528083801
, x: 0.5573590397834778
y: 0.4663642644882202
z: -1.3145148754119873
visibility: 0.9992757439613342
, x: 0.5785969495773315
y: 0.46770790219306946
z: -1.3149950504302979
visibility: 0.9992398023605347
, 

c:\Users\MANSI YAGNIK\facerecog\face\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\MANSI YAGNIK\facerecog\face\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Latency: 5.095346927642822
Latency: 1.3089511394500732
(120, 160, 3)
1/1 [==============================] - 0s 151ms/step
0.5949588221045218
Eyes not matched.
0.257615993292342
Eyes not matched.
0.185776747737708
Eyes not matched.
0
[x: 0.5058609247207642
y: 0.5613669157028198
z: -1.5434561967849731
visibility: 0.9995182156562805
, x: 0.5420113801956177
y: 0.4844328463077545
z: -1.4688138961791992
visibility: 0.9990299940109253
, x: 0.5646477937698364
y: 0.48360005021095276
z: -1.4696447849273682
visibility: 0.9991381168365479
, x: 0.5866146087646484
y: 0.4843306839466095
z: -1.4699164628982544
visibility: 0.9990492463111877
, x: 0.4658110737800598
y: 0.4904056191444397
z: -1.4578170776367188
visibility: 0.9989739656448364
, x: 0.4393700063228607
y: 0.4935876727104187
z: -1.4573816061019897
visibility: 0.9990790486335754
, x: 0.41528311371803284
y: 0.4978906214237213
z: -1.458186149597168
visibility: 0.9990201592445374
, x: 0.617768406867981
y: 0.516623854637146
z: -0.9033109545707703


c:\Users\MANSI YAGNIK\facerecog\face\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\MANSI YAGNIK\facerecog\face\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


(120, 160, 3)
1/1 [==============================] - 0s 151ms/step
0.585691898526862
Eyes not matched.
0.2474732308755838
Eyes not matched.
0.18291528093628484
Eyes not matched.
0
[x: 0.5097402334213257
y: 0.5711758732795715
z: -1.529176115989685
visibility: 0.9994574785232544
, x: 0.5466055870056152
y: 0.4939285218715668
z: -1.4545512199401855
visibility: 0.998935878276825
, x: 0.5692205429077148
y: 0.49316301941871643
z: -1.4552547931671143
visibility: 0.9990565180778503
, x: 0.5908772349357605
y: 0.49445268511772156
z: -1.4555270671844482
visibility: 0.9989079236984253
, x: 0.470700204372406
y: 0.5013188123703003
z: -1.4494855403900146
visibility: 0.998923122882843
, x: 0.44444528222084045
y: 0.5058811902999878
z: -1.4490199089050293
visibility: 0.9990572929382324
, x: 0.4202156066894531
y: 0.5116487145423889
z: -1.4497630596160889
visibility: 0.9990021586418152
, x: 0.6220341324806213
y: 0.5260246396064758
z: -0.8839119672775269
visibility: 0.998894989490509
, x: 0.3921129703521728

...

c:\Users\MANSI YAGNIK\facerecog\face\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\MANSI YAGNIK\facerecog\face\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


(120, 160, 3)
1/1 [==============================] - 0s 149ms/step
0.5915175398920982
Eyes not matched.
0.29236449852880564
Eyes not matched.
0.25180945176859426
Eyes not matched.
0
[x: 0.5350097417831421
y: 0.5899925827980042
z: -1.459160566329956
visibility: 0.9978633522987366
, x: 0.5629276037216187
y: 0.5266729593276978
z: -1.3656151294708252
visibility: 0.9969229102134705
, x: 0.5820968151092529
y: 0.5319041013717651
z: -1.3663609027862549
visibility: 0.99703049659729
, x: 0.6006451845169067
y: 0.5384972095489502
z: -1.366555094718933
visibility: 0.9968385100364685
, x: 0.5018206238746643
y: 0.5213624835014343
z: -1.379676342010498
visibility: 0.9972320795059204
, x: 0.47959667444229126
y: 0.5191792249679565
z: -1.3796168565750122
visibility: 0.997495710849762
, x: 0.45813870429992676
y: 0.5173199772834778
z: -1.3803337812423706
visibility: 0.9977210760116577
, x: 0.6314494609832764
y: 0.5781449675559998
z: -0.7803493142127991
visibility: 0.9965609312057495
, x: 0.4274899959564209

c:\Users\MANSI YAGNIK\facerecog\face\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\MANSI YAGNIK\facerecog\face\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


(120, 160, 3)
1/1 [==============================] - 0s 135ms/step
0.5730427558219369
Eyes not matched.
0.28148534597737057
Eyes not matched.
0.2592778432492835
Eyes not matched.
0
[x: 0.5450999140739441
y: 0.6004323959350586
z: -1.532767653465271
visibility: 0.9977722764015198
, x: 0.570469856262207
y: 0.5348654389381409
z: -1.4337154626846313
visibility: 0.9967384934425354
, x: 0.5875515937805176
y: 0.5388908386230469
z: -1.4344427585601807
visibility: 0.9968699216842651
, x: 0.6049093008041382
y: 0.5442816615104675
z: -1.4346802234649658
visibility: 0.996685802936554
, x: 0.508823812007904
y: 0.5294506549835205
z: -1.4557929039001465
visibility: 0.9970530271530151
, x: 0.48476535081863403
y: 0.5273694396018982
z: -1.4557095766067505
visibility: 0.9973461627960205
, x: 0.46187105774879456
y: 0.5258426666259766
z: -1.4565321207046509
visibility: 0.9975767135620117
, x: 0.631161093711853
y: 0.5812181830406189
z: -0.8336294889450073
visibility: 0.9963958859443665
, x: 0.427319198846817


c:\Users\MANSI YAGNIK\facerecog\face\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\MANSI YAGNIK\facerecog\face\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
